# Shopify Bulk Inventory Update Tool (SKU-Based)-Local Version with File Dialogs

Welcome to the local version of the Shopify Bulk Inventory Update Tool, designed to run on your local machine. This version enhances user experience by incorporating file dialogs for selecting and saving files, making it easier to navigate file systems without the need to manually enter file paths.

## How It Works

1. **Export your current Shopify Inventory:** Before running this notebook, export your current Shopify Inventory by navigating to: **Shopify Admin > Products > Inventory**. Then, **Choose Location** and Apply appropriate **filters** and then click **Export**. In the Export window, for Location, please select **One Location** at a time and for Quantity type, choose **All quantity types**.

2. **Run the Tool**: Execute the script. You will be prompted with file dialog boxes to select your Shopify Inventory Export File and your New Inventory File. These files should be in CSV format and contain your inventory data, including SKU information.

3. **File Dialogs**: Utilize the file dialogs to navigate your file system and select the appropriate files:
   - First, a dialog will appear asking you to select your Shopify file.
   - Next, you will be prompted to select your New Inventory file.

4. **Process and Export Inventory**: After selecting your files, the tool processes the inventory updates based on the new data. You will then be prompted to choose a location and name for the updated inventory file, which can be imported back into Shopify.

## Requirements

- This tool is designed for a local machine running JupyterLab, Jupyter Notebook, or as a standalone Python script.
- Ensure you have Python and pandas installed in your environment. tkinter, used for the file dialogs, is included with Python.
- Your inventory files must be in CSV format.

## Additional Notes

- This script provides a seamless way to update Shopify inventory using SKU information, filling a gap where Shopify's interface does not directly support such updates.
- To use this tool in Google Colab with interactive file upload and download capabilities, please see the corresponding file `Shopify_Inventory_Update_using_SKU_Colab.ipynb` in this repository.



In [ ]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd

# Initialize Tkinter
root = tk.Tk()
root.withdraw()  #hide the root window from appearing

# Function to open a file dialog and return the chosen file path
def browse_file_path(description):
    print(description)  # Optional, to print the action to the console
    filepath = filedialog.askopenfilename()  # Shows an "Open" dialog box
    return filepath

# Use the GUI to browse and select the Shopify and New Inventory files
print("Please select your Shopify file:")
shopify_file_path = browse_file_path("Select your Shopify file")
print("Please select your New Inventory file:")
new_inventory_file_path = browse_file_path("Select your New Inventory file")

# Reading the files into pandas DataFrames
if shopify_file_path and new_inventory_file_path:
    shopify_file = pd.read_csv(shopify_file_path)
    new_inventory = pd.read_csv(new_inventory_file_path)

    # Convert SKU columns to uppercase for case-insensitive comparison
    shopify_file['SKU'] = shopify_file['SKU'].str.upper()
    new_inventory['SKU'] = new_inventory['SKU'].str.upper()

    # Replace "ABC" with "XYZ" in the SKU columns to match both the SKU Files (if needed)
    shopify_file['SKU'] = shopify_file['SKU'].str.replace("ABC", "XYZ") # Change ABC & XYZ as needed
    new_inventory['SKU'] = new_inventory['SKU'].str.replace("ABC", "XYZ") # Change ABC & XYZ as needed

    # Update Shopify file with new inventory quantities
    for sku in shopify_file['SKU']:
        if sku in new_inventory['SKU'].values:
            quantity = new_inventory.loc[new_inventory['SKU'] == sku, 'Quantity'].values[0]
            shopify_file.loc[shopify_file['SKU'] == sku, 'Available'] = quantity
            shopify_file.loc[shopify_file['SKU'] == sku, 'On hand'] = quantity

    # Prompt user to save the updated Shopify inventory file
    output_filename = filedialog.asksaveasfilename(defaultextension=".csv",
                                                   filetypes=[("CSV files", "*.csv")],
                                                   title="Save the updated Shopify inventory as")
    if output_filename:
        shopify_file.to_csv(output_filename, index=False)
        print(f"Updated inventory has been saved to {output_filename}")
    else:
        print("File save cancelled.")
else:
    print("Operation cancelled or files not loaded.")
